# Show online metrics in jupyter

In [ ]:
%%capture
!pip install toloka-kit==0.1.15

In [ ]:
import toloka.metrics as metrics
from toloka.metrics.jupyter_dashboard import Chart, DashBoard
import toloka.client as toloka

toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION')
print(toloka_client.get_requester())

In [ ]:
new_dash = DashBoard(
    #toloka_client,
    # Will create chart for each element in list:
    [
        # Simple balance metric with default line names
        metrics.Balance(),
        # Assignments in pool. Only default lines with default names.
        # Adds pool private_name to chart name.
        metrics.AssignmentsInPool('26807107'),
        # Assignment events in pool. With self-configured line names.
        # Adds pool private_name to chart name
        # Joins all events that happens in one loop into one point.
        metrics.AssignmentEventsInPool('26807107', submitted_name='submitted', join_events=True),
        # Manually configured charts could contain several metrics and draw all its lines in one chart.
        # You must clearly specify chart name
        # Be careful, if you adds same metric type with default line names, you get pair of lines with same names.
        Chart(
            'Manualy configured chart',
            [metrics.AssignmentsInPool('26807107'), metrics.AssignmentsInPool('27262965'),]
        ),
        metrics.PoolCompletedPercentage('26807107'),
    ],

    update_seconds=2,  # just example. In realy dashboards it's better to drop this parameter
    header='My BEAUTY dashboard',
)

metrics.bind_client(new_dash.metrics, toloka_client)


In [ ]:
new_dash.run_dash()

___
### And you can run other cell, while this dashboard tracking metrics

In [ ]:
import time
for i in range(10):
    print(i)
    time.sleep(2)

### If you want to stop dashboard call this:

In [ ]:
new_dash.stop_dash()

### If you need several dashboard, create several instance of DashBoard and run it in several ports like:
```Python
    dash_for_requester1 = DashBoard(toloka_client1, [...])
    dash_for_requester2 = DashBoard(toloka_client2, [...])
```

And then run it in different cells. One:
```Python
    dash_for_requester1.run_dash(port='8081')
```

And another:
```Python
    dash_for_requester2.run_dash(port='8082')
```

### It's better to understand:
- This DashBoard useful only for fast and pretty online demonstarion in jupyter notebooks. For real monitoring system, please use `toloka.metric.MetricCollector`
- DashBoard don't draw something in jupyter (html or image or whatelse), it's post `IPython.lib.display.IFrame`. So if you save notebook and share .ipynb file, there is no dashboard images. Only big empty outup.
- DashBoard works with current instance, so you can not share it.

## Dash for several requesters
Create another client

In [ ]:
toloka_client2 = toloka.TolokaClient(input("Enter your another token:"), 'PRODUCTION')
print(toloka_client2.get_requester())

In [ ]:
new_dash2 = DashBoard(
    [
        metrics.Balance(toloka_client=toloka_client),
        metrics.Balance(toloka_client=toloka_client2),
        Chart(
            'Balance for both clients',
            [
                metrics.Balance(balance_name='first client', toloka_client=toloka_client),
                metrics.Balance(balance_name='second client', toloka_client=toloka_client2),
            ]
        )
    ],
    update_seconds=2,  # just example. In realy dashboards it's better to drop this parameter
    header='Dashboard for several clients',
)

# No client bindings, because we already set it in each metric
# metrics.bind_client(new_dash.metrics, toloka_client)

Lets run it on other port

In [ ]:
new_dash2.run_dash(port='8051')

Both dashboard run independently.
To stop second you must call:

In [ ]:
new_dash2.stop_dash()